In [1]:
#библиотеки
import pandas as pd
# Импортируем библиотеки Word2Vec
from gensim.models.word2vec import Word2Vec # Собственно модель.
from gensim.models.word2vec import LineSentence # Выравнивание текста по предложениям.
from gensim.models import KeyedVectors # Семантические вектора.
# На самом деле, нам потребуется только последняя.
import numpy as np # Вектора.
import pymorphy2
import re
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup

C:\Users\MI\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# загрузим данные, если new_try = True будет загружен изначальный набор данных, подгтовлен и сохранён
new_try = False

if new_try:
    df_train = pd.read_csv('data/train.csv', sep='\t', encoding='utf8')
    df_test = pd.read_csv('data/test.csv', sep='\t', encoding='utf8')
    
    df_train['description'] = [BeautifulSoup(text).get_text() for text in df_train['description'] ]
    df_test['description'] = [BeautifulSoup(text).get_text() for text in df_test['description'] ]
    
    df_train.to_csv('data/train_clear.csv', index=False)
    df_test.to_csv('data/test_clear.csv', index=False)
else:
    df_train = pd.read_csv('data/train_clear.csv', sep=',', encoding='utf8')
    df_test = pd.read_csv('data/test_clear.csv', sep=',', encoding='utf8')

In [3]:
def prep_str(text):
    text = text.lower()
    return [a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]

In [4]:
%%time
df_train['name'] = df_train['name'].apply(lambda x: prep_str(x))
df_train['description'] = df_train['description'].apply(lambda x: prep_str(x))
df_test['name'] = df_test['name'].apply(lambda x: prep_str(x))
df_test['description'] = df_test['description'].apply(lambda x: prep_str(x))


Wall time: 25.3 s


In [5]:
default_size = 100

def create_model(data):
    model = Word2Vec(data, size=default_size, min_count=10, workers=4)
    #model.build_vocab(data)
    #model.train(data, total_examples=model.corpus_count, epochs=model.epochs)
    return model

In [24]:
%%time
model_name = create_model(np.append(df_test.name.values, df_train.name.values))


Wall time: 3.84 s


In [25]:
%%time
model_description = create_model(np.append(df_test.description.values, df_train.description.values))

Wall time: 2min 24s


In [51]:
def get_sentence_vec(model, words):
    return np.mean([model[w] for w in words if w in model] or [np.zeros(default_size)], axis=0)

def get_sentence_list_vec(model, texts):
    return np.array([get_sentence_vec(model, d) for d in texts])

In [52]:
%%time

df_train_name_vec = get_sentence_list_vec(model_name, df_train.name.values)
df_train_desc_vec = get_sentence_list_vec(model_description, df_train.description.values)

df_test_name_vec = get_sentence_list_vec(model_name, df_test.name.values)
df_test_desc_vec = get_sentence_list_vec(model_description, df_test.description.values)

C:\Users\MI\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
C:\Users\MI\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Wall time: 4min 49s


In [53]:
df_train_full_vec = np.hstack((df_train_name_vec, df_train_desc_vec))
df_test_full_vec = np.hstack((df_test_name_vec, df_test_desc_vec))

#df_train_full_vec = pd.DataFrame()
#df_train_full_vec['name'] = df_train_name_vec
#df_train_full_vec['desc'] = df_train_desc_vec
#df_test_full_vec = pd.DataFrame()
#df_test_full_vec['name'] = df_test_name_vec
#df_test_full_vec['desc'] = df_test_desc_vec


In [54]:
X_train, X_test, y_train, y_test = train_test_split(df_train_full_vec, df_train['target'], test_size=0.3, random_state=42)

In [55]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#C_param_range = [1, 10, 100, 200, 400, 500, 800, 1000]
C_param_range = [1]
models = []
#models.append({'model': DecisionTreeClassifier(random_state=13), 'predict': 0, 'name': 'DTC'})
#models.append({'model': KNeighborsClassifier(), 'predict': 0, 'name': 'RF'})
for i in C_param_range:
    models.append({'model': LogisticRegression(penalty='l2', solver='liblinear', random_state=42, C = i), 'predict': 0, 'name': 'LR ' + str(i)})

In [56]:
for m in models:
    m['model'].fit(X_train, y_train)
    m['predict'] = m['model'].predict(X_test)
    m['predict_probe'] = m['model'].predict_proba(X_test)

In [57]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve

## Your Code Here
data = []
for m in models:
    data.append({
        'name': m['name'], 
        'precision': precision_score(y_test, m['predict']), 
        'recall': recall_score(y_test, m['predict']), 
        'accuracy': accuracy_score(y_test, m['predict']), 
        'roc_auc': roc_auc_score(y_test, m['predict_probe'][:,1]),
    })
    
data_df = pd.DataFrame(data)

data_df

,accuracy,name,precision,recall,roc_auc
0,0.948967,LR 1,0.941675,0.950103,0.98482


In [58]:
result = pd.DataFrame(df_test['id'])

best_model = models[0]
result['target'] = best_model['model'].predict_proba(df_test_full_vec)[:,1] 

result.to_csv('result_IKA.csv', index=False)

In [59]:
#todo
# данные надо объединить при обучении - done
# надо попробовать mean без осей - roc_auc = 0.805372
# надо попробовать pymorphy и готовый словарь w2v - позже


In [98]:
# поищем похожие вакансии
target_index = 3
eq_index = 3
target = df_train_full_vec[target_index]

print(df_train.iloc[[target_index]])

for eq_index in range(2,200000):
    eq = df_train_full_vec[eq_index]
    cosine_similarity = np.dot(target, eq)/(np.linalg.norm(target)* np.linalg.norm(eq))
    #print(eq_index)
    if (cosine_similarity > 0.90):
        print('-' * 10)
        print(cosine_similarity)
        print(df_train.iloc[[eq_index]])

   id           name                                        description  \
3   3  [фрезеровщик]  [условия, на, работу, вахтовым, методом, в, г,...   

   target  
3       0  
----------
0.9999999999999999
   id           name                                        description  \
3   3  [фрезеровщик]  [условия, на, работу, вахтовым, методом, в, г,...   

   target  
3       0  
----------
0.9590207263345277
      id      name                                        description  target
221  221  [токарь]  [на, работу, вахтовым, методом, в, г, калуга, ...       0
----------
0.9261595349603486
      id          name                                        description  \
346  346  [сверловщик]  [обязанности, механическая, обработка, деталей...   

     target  
346       0  
----------
0.9303576288259723
      id           name                                        description  \
450  450  [фрезеровщик]  [работа, вахтовым, методом, тульская, область,...   

     target  
450       0  
------

C:\Users\MI\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


----------
0.9999999999999999
          id           name  \
15400  15400  [фрезеровщик]   

                                             description  target  
15400  [условия, на, работу, вахтовым, методом, в, г,...       0  
----------
0.9261595349603486
          id          name                                        description  \
15954  15954  [сверловщик]  [обязанности, механическая, обработка, деталей...   

       target  
15954       0  
----------
0.9363260882234322
          id           name  \
15983  15983  [фрезеровщик]   

                                             description  target  
15983  [обязанности, механическая, обработка, деталей...       0  
----------
0.9491636749197523
          id           name  \
16221  16221  [фрезеровщик]   

                                             description  target  
16221  [на, работу, вахтовым, методом, в, тульскую, о...       0  
----------
0.902206404745372
          id      name                                        des

----------
0.923637736010446
          id           name  \
57778  57778  [фрезеровщик]   

                                             description  target  
57778  [обязанности, работа, на, универсально-фрезерн...       0  
----------
0.9937524686330017
          id           name  \
57849  57849  [фрезеровщик]   

                                             description  target  
57849  [на, работу, вахтовым, методом, в, г, калуга, ...       0  
----------
0.9999999999999999
          id           name  \
58718  58718  [фрезеровщик]   

                                             description  target  
58718  [условия, на, работу, вахтовым, методом, в, г,...       0  
----------
0.923637736010446
          id           name  \
60346  60346  [фрезеровщик]   

                                             description  target  
60346  [обязанности, работа, на, универсально-фрезерн...       0  
----------
0.9999999999999999
          id           name  \
60465  60465  [фрезеровщик]   

 

----------
0.9303576288259723
          id           name  \
96036  96036  [фрезеровщик]   

                                             description  target  
96036  [работа, вахтовым, методом, тульская, область,...       0  
----------
0.9937524686330017
          id           name  \
96255  96255  [фрезеровщик]   

                                             description  target  
96255  [на, работу, вахтовым, методом, в, г, калуга, ...       0  
----------
0.9012449151353311
          id                  name  \
98364  98364  [токарь-карусельщик]   

                                             description  target  
98364  [обязанности, токарно-карусельный, ух, стоечны...       0  
----------
0.9491636749197523
          id           name  \
98933  98933  [фрезеровщик]   

                                             description  target  
98933  [на, работу, вахтовым, методом, в, тульскую, о...       0  
----------
0.9491636749197523
            id           name  \
101427  101427 

----------
0.9590207263345277
            id      name                                        description  \
143113  143113  [токарь]  [на, работу, вахтовым, методом, в, г, калуга, ...   

        target  
143113       0  
----------
0.902206404745372
            id      name                                        description  \
144103  144103  [токарь]  [обязанности, выполнение, токарных, работ, на,...   

        target  
144103       0  
----------
0.9375862704178685
            id           name  \
146337  146337  [фрезеровщик]   

                                              description  target  
146337  [работа, вахтовым, методом, тульская, область,...       0  
----------
0.9999999999999999
            id           name  \
147329  147329  [фрезеровщик]   

                                              description  target  
147329  [условия, на, работу, вахтовым, методом, в, г,...       0  
----------
0.902206404745372
            id      name                                    

----------
0.9024186625403879
            id                  name  \
188631  188631  [токарь-карусельщик]   

                                              description  target  
188631  [работа, вахтовым, методом, г, калуга, вахта, ...       0  
----------
0.9012449151353311
            id                  name  \
189419  189419  [токарь-карусельщик]   

                                              description  target  
189419  [обязанности, токарно-карусельный, ух, стоечны...       0  
----------
0.9261595349603486
            id          name  \
191748  191748  [сверловщик]   

                                              description  target  
191748  [обязанности, механическая, обработка, деталей...       0  
----------
0.9590207263345277
            id      name                                        description  \
194925  194925  [токарь]  [на, работу, вахтовым, методом, в, г, калуга, ...   

        target  
194925       0  
----------
0.923637736010446
            id         

      id                            name  \
100  100  [руководитель, отдела, продаж]   

                                           description  target  
100  [компания, объявляет, о, вакансии, руководител...       1  
